# DR16Q bitmask testing
DR16Q_v3.fits appeared to have the RQS bitmasks set incorrectly compared to spAll-v5_13_0.  This notebook investigates this issue more rigorously in DR16Q_v4.fits which is current.

In order to run this, you'll need DR16Q_v4.fits and spAll-v5_13_0.fits, which you can obtain from these websites:
https://www.sdss.org/dr16/algorithms/qso_catalog/

https://data.sdss.org/sas/dr16/eboss/spectro/redux/v5_13_0/  (<-- Vorsicht!  This is 14.8GB.)

From Lyke et al. (2020) Table 1, it appears that the RQS bitmasks may have been excluded because they were not listed in the .par file when the catalog was made.  This is still confusing, because querying them does return something.  This cut in bitmasks is reflected in the Python code that pulls quasars out of the spAll file to make the superset.  It is located here:

https://github.com/bradlyke/dr16q/blob/master/spAll_class_full.py

# Setup

In [1]:
# import block
import numpy as np
from astropy.io import fits
from pydl.pydlutils.sdss import sdss_flagval as flagval
from pydl.pydlutils.sdss import set_maskbits

In [2]:
# setup
data_dir = '/Users/runnojc1/Research/data/sdss/dr16/'

# DR16Q_v4.fits bitmask check

In [3]:
# read in DR16Q_v4.fits
# https://www.sdss.org/dr16/algorithms/qso_catalog/
dr16_hdu = fits.open(data_dir+'DR16Q_v4.fits')
hdr      = dr16_hdu[1].header
dr16     = dr16_hdu[1].data

In [4]:
# set the RQS bitmasks
# https://www.sdss.org/dr16/algorithms/bitmasks/#EBOSS_TARGET2
TDSS_RQS1  = 2**34
TDSS_RQS2  = 2**35
TDSS_RQS3  = 2**36
TDSS_RQS2v = 2**37
TDSS_RQS3v = 2**38

# pull the bitmask
eboss_target2 = (dr16['EBOSS_TARGET2'])

# check them
rqs = np.where((eboss_target2 & TDSS_RQS1) | (eboss_target2 & TDSS_RQS2) | (eboss_target2 & TDSS_RQS3) | (eboss_target2 & TDSS_RQS2v) | (eboss_target2 & TDSS_RQS3v))[0]

In [5]:
# try again with slightly different syntax
# (it is ultimately not very different)

# note that the pydl flagval function grabs this .par file:
# https://svn.sdss.org/public/repo/sdss/idlutils/tags/v5_5_24/data/sdss/sdssMaskbits.par
# which does not have the RQS flags.  Use a more recent one instead:
# https://svn.sdss.org/public/repo/sdss/idlutils/tags/v5_5_33/data/sdss/sdssMaskbits.par

# also, the pydl function automatically capitalizes your input flags
# this means that it can't find TDSS_RQS2v and TDSS_RQS3v

# I edited pydl/pydlutils/sdss.py to default to v5_5_33 and not to capitalize flags

et2_flag_names = np.array(['TDSS_RQS1','TDSS_RQS2','TDSS_RQS3','TDSS_RQS2v','TDSS_RQS3v'])
et2 = flagval('EBOSS_TARGET2',et2_flag_names)
rqs = np.where((dr16['EBOSS_TARGET2']&et2))[0]

In [6]:
("Count {0} objects with an RQS bitmask set in DR16Q_v4.fits.").format(np.shape(rqs)[0])

'Count 80129 objects with an RQS bitmask set in DR16Q_v4.fits.'

# spAll_v5_13_0.fits bitmask check

In [2]:
# read in spAll-v5_13_0
# https://data.sdss.org/sas/dr16/eboss/spectro/redux/v5_13_0/
data_dir = '/Users/runnoejc/Desktop/Research/2019_NSF_AAG/'

In [3]:
spall_hdu = fits.open(data_dir+'spAll-v5_13_0.fits.gz')
spall_header = spall_hdu[1].header
spall = spall_hdu[1].data

In [4]:
# set the RQS bitmasks
# https://www.sdss.org/dr16/algorithms/bitmasks/#EBOSS_TARGET2
TDSS_RQS1  = 2**34
TDSS_RQS2  = 2**35
TDSS_RQS3  = 2**36
TDSS_RQS2v = 2**37
TDSS_RQS3v = 2**38

# pull the bitmask
eboss_target2 = (spall['EBOSS_TARGET2'])

# check them
rqs = np.where((eboss_target2 & TDSS_RQS1) | (eboss_target2 & TDSS_RQS2) | (eboss_target2 & TDSS_RQS3) | (eboss_target2 & TDSS_RQS2v) | (eboss_target2 & TDSS_RQS3v))[0]


In [6]:
("Count {0} objects with an RQS bitmask set in spAll-v5_13_0.fits.").format(np.shape(rqs)[0])

'Count 16509 objects with an RQS bitmask set in spAll-v5_13_0.fits.'